In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install sklearn
!pip install h5py

In [ ]:
# Name of the file : Valueassign.py
# Brief : Extracting the values from the word file file and removing the NAN values of the columns. Then assgining the words to a unique value.
# Date : 27 March 2021
# Important functions : pd.read_csv() - Using panda function to read the word file and give headers to all the columns
#                       lamda x:x function and apply() function - USing both to assign the values of the last three columns and combine the words


import pandas as pd
# Delimiter the word file 
a = pd.read_csv('/content/drive/MyDrive/datadir/words_4.txt',names=["col1","col2","col3","col4","col5","col6","col7", "col8","col9","col10","col11"],low_memory=False, delimiter=" ")
# Printing the txt file length to see total words and cross check with the number of images
print(len(a))

# b = a[['col9', 'col10','col11']]
# print(b) 

# Combining the rows of the columns using apply and lambda functions
a['combined'] = a[['col9', 'col10','col11']].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

# Removing the NaN values from the combined columns
for i in range(len(a['combined'])):
    a['combined'][i] = a['combined'][i].replace("nan", "")
    a['combined'][i] = a['combined'][i].replace(" ", "")

a['combined'].head()

from sklearn.preprocessing import LabelEncoder
# Lable Encoding
le = LabelEncoder()
a['combined_2'] = le.fit_transform(a['combined'])
# getting unique value to the combine cols
uniqueValues = a['combined_2'].unique()
print(len(uniqueValues))
outputCount = len(uniqueValues)

In [ ]:
# Name of the file : model.py
# Brief : Loading data and training the dataset in small batches
# Date : 4 April 2021
# Important functions : models.Sequential() - Grouping linear stack of layers
#                       model.compile() - Configure model for training
#                       model.summary() - Summarizing the model and its parameters
#                       model.load_weights() - Loading weights of the model
#                       flow_from_directory() - Takes the path to a directory & generates batches of augmented data
#                       model.fit() - Trains the model for a fixed number of epochs
#                       model.save() - save the model


import tensorflow as tf
from tensorflow.keras import models,layers
import keras
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
img_height = 256
img_width = 256
batch_size = 18
start = 0
end=batch_size
datagen = tf.keras.preprocessing.image.ImageDataGenerator()


model = models.Sequential()
model.add(layers.Conv2D(128, (3,3), activation='relu', input_shape=(img_height,img_width, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3,3), activation='relu'))
model.add(layers.Flatten())

model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(outputCount))
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

model.load_weights('/content/drive/MyDrive/data_dir/model_history.h5')

for j in range(100):
    print("Epochs "+str(j))
    start = 0
    end = batch_size
    # Generating batches of augmented data
    train_ds = datagen.flow_from_directory("/content/drive/MyDrive/imgs", batch_size=batch_size,shuffle=False)
    for i in range(int(train_ds.samples / batch_size) ):
        # print(a['col1'][start:end])
        x_train = train_ds.next()[0]
        y_train = a['combined_2'][start:end]
        history = model.fit(x_train,y_train, epochs=1)
        start = start + batch_size
        end = end + batch_size
      
model.save('/content/drive/MyDrive/data_dir/model_history.h5')

In [ ]:
# Name of the file : output.py
# Brief : 
# Date : 13 April 2021
# Important functions : np.argmax() - Returns the indices of the maximum values along an axis of combine_2.

import cv2
img = cv2.imread("/content/test.png")
dim = (img_width, img_height)
# resize image
img = cv2.resize(img, dim).reshape(1,256,256,3)
i = 0
for j in range(len(a.index[a["combined_2"] == np.argmax(model.predict(img).tolist())])):
        if i == 0:
          print(f' Prediction {a["combined"][j]}')
          i = 1
